In [13]:
import pandas as pd
import folium

# Carrega as primeiras 3 análises como exemplo
df = pd.read_csv('/home/lucas/dev/Security-Monitor/src/data/reports/relatorio_cpted_multiplas_analises.csv').head(3)

# Mockup de coordenadas
coords = [
    (-15.7939, -47.8828),
    (-15.7990, -47.8650),
    (-15.7600, -47.9000)
]
df['lat'] = [c[0] for c in coords]
df['lon'] = [c[1] for c in coords]

def cor_icone(risco):
    risco = risco.lower()
    if 'fraco' in risco:
        return 'red'
    elif 'moderado' in risco:
        return 'orange'
    elif 'forte' in risco:
        return 'green'
    else:
        return 'gray'

m = folium.Map(location=[-15.7939, -47.8828], zoom_start=12)

for _, row in df.iterrows():
    cor = cor_icone(row['indice_cpted_geral'])
    # monta o HTML do popup com detalhes colapsáveis
    popup_html = f"""
    <div style="width:250px;">
      <strong>{row['titulo_analise']}</strong><br>
      Índice CPTED: <em>{row['indice_cpted_geral']}</em><br>
      <details>
        <summary style="cursor:pointer;color:blue;">Mais informações</summary>
        <ul style="padding-left:1em;margin:0;">
          <li><strong>Id do usuário:</strong> {row.get('id','—')}</li>
          <li><strong>Data da Análise:</strong> {row.get('data_analise','—')}</li>
          <li><strong>Resumo Executivo:</strong> {row.get('resumo_executivo','—')}</li>
          <li><strong>Recomendação:</strong> {row.get('recomendacoes','—')}</li>

        </ul>
      </details>
    </div>
    """
    popup = folium.Popup(popup_html, max_width=300)
    folium.Marker(
        [row['lat'], row['lon']],
        popup=popup,
        icon=folium.Icon(color=cor)
    ).add_to(m)

# salva e exibe
m.save('mapa_cpted_df.html')

m


In [16]:
import pandas as pd
import folium

def gerar_mapa_cpted_html(csv_path: str,
                          coords: list[tuple[float,float]],
                          zoom_start: int = 12) -> str:
    """
    Gera um mapa Folium como string HTML contendo todos os assets inline
    (CSS/JS) para injetar direto num template ou retornar numa API.
    """
    # Lê CSV e injeta coords
    df = pd.read_csv(csv_path).head(len(coords))
    df['lat'], df['lon'] = zip(*coords)

    # Função de cor (fraco = alto risco, etc)
    def cor_icone(risco: str) -> str:
        r = risco.lower()
        if 'fraco' in r:      return 'red'
        if 'moderado' in r:   return 'orange'
        if 'forte' in r:      return 'green'
        return 'gray'

    # Cria mapa centrado no primeiro ponto
    centro = [df.iloc[0]['lat'], df.iloc[0]['lon']]
    m = folium.Map(location=centro, zoom_start=zoom_start)

    # Adiciona marcadores com <details>
    for _, row in df.iterrows():
        popup_html = f"""
        <div style="width:250px;">
          <strong>{row['titulo_analise']}</strong><br>
          Índice CPTED: <em>{row['indice_cpted_geral']}</em><br>
          <details>
            <summary style="cursor:pointer;color:blue;">Ver mais</summary>
            <ul style="padding-left:1em;margin:0;">
              <li><strong>Data:</strong> {row.get('data_analise','—')}</li>
              <li><strong>Resumo:</strong> {row.get('resumo_executivo','—')}</li>
              <li><strong>Recomendações:</strong> {row.get('recomendacao','—')}</li>
            </ul>
          </details>
        </div>
        """
        folium.Marker(
            [row['lat'], row['lon']],
            popup=folium.Popup(popup_html, max_width=300),
            icon=folium.Icon(color=cor_icone(row['indice_cpted_geral']))
        ).add_to(m)

    # Retorna o HTML completo como string
    return m.get_root().render()

# Exemplo de uso:
if __name__ == "__main__":
    csv_file = "/home/lucas/dev/Security-Monitor/src/data/reports/relatorio_cpted_multiplas_analises.csv"
    coordenadas = [
        (-15.7939, -47.8828),
        (-15.7990, -47.8650),
        (-15.7600, -47.9000)
    ]
    html_map = gerar_mapa_cpted_html(csv_file, coordenadas)

    print(html_map[:200] + '...')


import tempfile
import webbrowser

# Gere a string
html_map = gerar_mapa_cpted_html(csv_file, coordenadas)

# Cria um arquivo temporário
with tempfile.NamedTemporaryFile("w", suffix=".html", delete=False) as f:
    f.write(html_map)
    temp_path = f.name

# Abre no navegador padrão
webbrowser.open(f"file://{temp_path}")


<!DOCTYPE html>
<html>
<head>
    
    <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
    <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>
    ...


True

Abrindo em uma sessão de navegador existente.
